# 패키지

In [1]:
import os                           # 운영체제
import tensorflow as tf             # 텐서플로
import pandas as pd                 # 판다스
import matplotlib.pyplot as plt     # 그래프 도구
import my_models as mymod           # 케라스 사용자 모델
import my_dataset as myds           # 데이터셋 생성기

/Users/chorokmoo/_MyDev/_Anaconda/anaconda3/envs/new2/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# GPU 설정

In [2]:
# 사용 가능한 모든 GPU 리스트
gpus = tf.config.list_physical_devices('GPU')

# 사용할 프로세서 선택 : GPU
if gpus:
    try: tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: print(e)

# 데이터셋

In [3]:
ds = myds.make()
ds_validation = ds.take(320).batch(32).prefetch(tf.data.experimental.AUTOTUNE)  # 훈련 데이터셋
ds_train = ds.skip(320).batch(32)                                               # 검증 데이터셋

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

전체 분류
--------------------------------
df_all          : (22617, 12)

세부 분류
--------------------------------
df_pitch        : (22617, 11)
df_note         : (22617, 14)
df_accidental   : (22617, 5)
df_articulation : (22617, 9)
df_dynamic      : (22617, 18)
df_octave       : (22617, 8)
df_ornament     : (22617, 7)
df_repetition   : (22617, 12)
df_clef         : (22617, 9)
df_key          : (22617, 14)
df_measure      : (22617, 6)
df_rest         : (22617, 15)
df_time         : (22617, 2)


# 콜백 함수

In [4]:
# 모델 주소
model_dir = os.path.join('.', 'models', 'msrm.h5')

# 최상의 모델 저장
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_dir, save_best_only=True)
cb_early_stop = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

# 모델 생성 및 학습

In [5]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir): model = tf.keras.models.load_model(model_dir) # 모델 불러오기
    else                        : model = mymod.msrm(myds.t_dfs)                # 모델 생성

    # 모델 학습 설정
    model.compile(
        optimizer='adam',
        loss=[
            'binary_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy'
        ],
        metrics=['accuracy']
    )

    # 모델 훈련 및 저장
    history = model.fit(ds_train, epochs=10, initial_epoch=0, validation_data=ds_validation, callbacks=[cb_checkpoint, cb_early_stop])

    # 모델 저장
    model.save(model_dir)

Epoch 1/10


2024-07-18 16:22:02.768224: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


697/697 [==============================] - 672s 936ms/step - loss: 9.4754 - all_loss: 0.1766 - pitch_loss: 2.3881 - note_loss: 2.6266 - accidental_loss: 0.4306 - articulation_loss: 0.7206 - dynamic_loss: 1.6952 - octave_loss: 0.7530 - ornament_loss: 0.6554 - repetition_loss: 0.0207 - clef_loss: 9.2501e-04 - key_loss: 0.0017 - measure_loss: 0.0041 - rest_loss: 0.0019 - time_loss: 7.1702e-05 - all_accuracy: 0.9501 - pitch_accuracy: 0.0889 - note_accuracy: 0.0681 - accidental_accuracy: 0.0893 - articulation_accuracy: 0.0486 - dynamic_accuracy: 0.0317 - octave_accuracy: 0.0430 - ornament_accuracy: 0.1110 - repetition_accuracy: 0.0868 - clef_accuracy: 0.2167 - key_accuracy: 0.4444 - measure_accuracy: 0.8251 - rest_accuracy: 4.4849e-05 - time_accuracy: 4.4849e-05 - val_loss: 9.3390 - val_all_loss: 0.1015 - val_pitch_loss: 2.3848 - val_note_loss: 2.6199 - val_accidental_loss: 0.4030 - val_articulation_loss: 0.7081 - val_dynamic_loss: 1.6798 - val_octave_loss: 0.7278 - val_ornament_loss: 0.680

# 학습 결과

In [ ]:
# 훈련 결과 그리기
pd.DataFrame(history.history).plot(figsize=(20, 10))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()